# ***"Методи прийняття рішень"***

## Завдання

### **Мета**

Маючи вхідний набір даних, опрацювати його, розв’язати задачу прогнозування ціни оренди нерухомості, описати свої дії у презентації.

Рішення міститиме **Jupyter Notebook** та презентацію. Тут описані вимоги до **Jupyter Notebook**.

### **Вимоги**
Розв’язок має складатися з 3-х основних частин:
1. Аналіз та підготовка даних.
2. Моделювання.
3. Аналіз результатів.


**Частина 1. Аналіз та підготовка даних**

- Проаналізуй діапазони значень, їх середні та дисперсію, розподіли, кореляції, наявність пропущених значень, типи ознак.
- Перевір на наявність пропущених значень. За наявності заповни їх або видали рядки з пропусками.
- За наявності бінарних ознак заміни значення цих ознак на 0 та 1.
- За потреби певні дані можуть бути видалено або модифіковано.

**Частина 2. Моделювання**

- Створи як мінімум 2 різні регресійні моделі для прогнозування цільової змінної “Rent”. 
- Моделі вважаються різними, якщо вони базуються на різних даних або є різними типами моделей.
- Оціни якість моделей як мінімум за 2 метриками.
- Оцінка має бути проведена або на тестовій вибірці, або за допомогою крос-валідації.

**Частина 3. Аналіз результатів**

- Для всіх моделей побудуй графіки порівняння справжніх та спрогнозованих даних.
- Зроби висновки, на яких прикладах кожна модель більше помиляється.
- Зроби порівняння якості моделей. Напиши висновки.

**Додатково**
- Проаналізуй ознаки на мультиколінеарність (Variance inflation factor).
- Знайди оптимальні значення параметрів моделей, наприклад, за допомогою алгоритму GridSearch.
- Проаналізуй важливість ознак (feature importance).
- Побудуй графік залишків (residuals plot).

### **Джерело даних** 

[**Датасет "House Rent Prediction Dataset”:**](https://docs.google.com/spreadsheets/d/1OPnEAT64Patnj_Ifhwn_pM1c15rsBNIoFrtz38A1_W4/edit?gid=881312933#gid=881312933)

https://docs.google.com/spreadsheets/d/1OPnEAT64Patnj_Ifhwn_pM1c15rsBNIoFrtz38A1_W4/edit?gid=881312933#gid=881312933

## **ЧАСТИНА 1: *Аналіз та підготовка даних***

### Отримання даних з посилання на Google Sheets

У зв'язку з тим, що початковий документ, посилання на який дано в завданні, містить різні аркуші, було вирішено почати підготовку даних з пошуку та завантаження саме того аркушу, що містить у назві ключові слова. Для цього було розроблено скрипт на Python, який автоматизує цей процес.

**Функція `find_csv_link_by_keywords`**

Ця функція приймає URL на Google Sheets документ та список ключових слів. Вона здійснює такі кроки:
1. Завантажує HTML-код сторінки за вказаним URL.
2. Парсить HTML-код для пошуку назв аркушів у документі.
3. Перевіряє, чи містять назви аркушів вказані ключові слова.
4. Якщо знаходиться відповідний аркуш, функція формує URL для експорту цього аркуша у форматі `.csv` і повертає цей URL. Якщо аркуш не знайдено, повертається повідомлення про помилку.

**Функція `get_spreadsheet_id`**

Ця допоміжна функція витягує ідентифікатор документа з URL-адреси Google Sheets. Вона використовується в `find_csv_link_by_keywords` для правильного формування URL на `.csv` документ.

**Функція `is_csv_url`**

Перевіряє, чи вказує передана URL-адреса на `.csv` документ. Вона надсилає HTTP-запит типу HEAD до URL і аналізує заголовок `Content-Type`. Якщо заголовок вказує на тип `text/csv`, функція повертає `True`, інакше `False`. Це дозволяє уникнути помилок при завантаженні даних з неправильного формату.


In [169]:
import urllib.parse

import pandas as pd
import requests
from bs4 import BeautifulSoup


def find_csv_link_by_keywords(url: str, keywords: list) -> str:
    """
    Функція приймає URL на Google Sheets документ і список ключових слів.
    Повертає посилання на .csv-документ, що містить дані, пов'язані з цими ключовими словами.

    Parameters
    ----------
    url : str
        URL на Google Sheets документ.
    keywords : list of str
        Список ключових слів для пошуку.

    Returns
    -------
    str
        URL на .csv файл або повідомлення про помилку.
    """
    try:
        # Отримуємо HTML сторінки за допомогою HTTP запиту
        response = requests.get(url)

        # Перевіряємо, чи успішний запит (код 200 означає успіх)
        if response.status_code != 200:
            return f"Помилка доступу до URL: {response.status_code}"

        # Використовуємо BeautifulSoup для парсингу HTML-коду сторінки
        soup = BeautifulSoup(response.text, "html.parser")

        # Збираємо всі назви аркушів (вони зберігаються в <div> з класом 'docs-sheet-tab')
        sheet_titles = [
            div.get_text() for div in soup.find_all("div", class_="docs-sheet-tab")
        ]

        # Перебираємо всі назви аркушів і перевіряємо, чи містять вони ключові слова
        for title in sheet_titles:
            for keyword in keywords:
                # Якщо ключове слово є в назві аркуша (незалежно від регістру)
                if keyword.lower() in title.lower():
                    # Формуємо URL для експорту аркуша у форматі .csv
                    csv_url = (
                        f"https://docs.google.com/spreadsheets/d/"
                        f"{get_spreadsheet_id(url)}/gviz/tq?tqx=out:csv&sheet="
                        f"{urllib.parse.quote(title)}"
                    )
                    return csv_url

        # Якщо жоден аркуш не відповідає ключовим словам, повертаємо повідомлення
        return "Не знайдено аркушів, що відповідають ключовим словам."

    except Exception as e:
        # Якщо виникла будь-яка інша помилка, повертаємо її опис
        return f"Сталася помилка: {str(e)}"


def get_spreadsheet_id(url: str) -> str:
    """
    Функція для витягування ID документа з URL.

    Parameters
    ----------
    url : str
        URL на Google Sheets документ.

    Returns
    -------
    str
        ID документа.

    Raises
    ------
    ValueError
        Якщо не вдалося витягти ID документа з URL.
    """
    # Парсимо URL для отримання окремих частин
    parsed_url = urllib.parse.urlparse(url)
    path_parts = parsed_url.path.split("/")

    # Ідентифікатор документа знаходиться між 'd' і наступним елементом у URL
    if "d" in path_parts:
        return path_parts[path_parts.index("d") + 1]
    else:
        # Якщо структура URL не відповідає очікуваній, генеруємо помилку
        raise ValueError("Не вдалося витягти ID документа з URL.")


def is_csv_url(url: str) -> bool:
    """
    Функція перевіряє, чи вказує передана URL-адреса на .csv-документ.

    Parameters
    ----------
    url : str
        URL для перевірки.

    Returns
    -------
    bool
        True, якщо URL вказує на .csv документ, інакше False.
    """
    try:
        # Здійснюємо запит до URL
        response = requests.head(url, allow_redirects=True)

        # Перевіряємо заголовки, зокрема Content-Type
        content_type = response.headers.get("Content-Type", "")

        # Повертаємо True, якщо Content-Type вказує на CSV
        return "text/csv" in content_type
    except Exception as e:
        print(f"Помилка при перевірці URL: {str(e)}")
        return False


# Тестування функції
keywords = ["house", "rent"]
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1OPnEAT64Patnj_Ifhwn_pM1c15rsBNIoFrtz38A1_W4/edit?gid=881312933#gid=881312933"
csv_link = find_csv_link_by_keywords(spreadsheet_url, keywords)
print(f'Результат роботи функції "find_csv_link_by_keywords": {csv_link}')

print(f'Перевірка отриманого посилання: {csv_link.startswith("http")}')
# Перевірка, чи URL вказує на .csv файл
if is_csv_url(csv_link):
    # Завантажуємо дані у DataFrame за допомогою pandas
    df = pd.read_csv(csv_link)
    # Виводимо перші 5 рядків таблиці для перевірки
    print(df.head())
else:
    print("Помилка: не вдалося завантажити дані.")



Результат роботи функції "find_csv_link_by_keywords": https://docs.google.com/spreadsheets/d/1OPnEAT64Patnj_Ifhwn_pM1c15rsBNIoFrtz38A1_W4/gviz/tq?tqx=out:csv&sheet=House_Rent_Dataset
Перевірка отриманого посилання: True
    Posted On  BHK   Rent  Size            Floor    Area Type  \
0  2022-05-18    2  10000  1100  Ground out of 2   Super Area   
1  2022-05-13    2  20000   800       1 out of 3   Super Area   
2  2022-05-16    2  17000  1000       1 out of 3   Super Area   
3  2022-07-04    2  10000   800       1 out of 2   Super Area   
4  2022-05-09    2   7500   850       1 out of 2  Carpet Area   

              Area Locality     City Furnishing Status  Tenant Preferred  \
0                    Bandel  Kolkata       Unfurnished  Bachelors/Family   
1  Phool Bagan, Kankurgachi  Kolkata    Semi-Furnished  Bachelors/Family   
2   Salt Lake City Sector 2  Kolkata    Semi-Furnished  Bachelors/Family   
3               Dumdum Park  Kolkata       Unfurnished  Bachelors/Family   
4        

### Аналіз та підготовка даних

#### Завантаження та попередній огляд даних

Спершу завантажимо набір даних і подивимося на перші кілька рядків, щоб зрозуміти структуру та зміст даних.
Для завантаження скористаємось функцією `find_csv_link_by_keywords`, створеною раніше.

In [170]:
keywords = ["house", "rent"]
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1OPnEAT64Patnj_Ifhwn_pM1c15rsBNIoFrtz38A1_W4/edit?gid=881312933#gid=881312933"
csv_link = find_csv_link_by_keywords(spreadsheet_url, keywords)

# Виведення перших 5 рядків (за замовчуванням) набору даних
print("Перші кілька рядків даних:")
print(df.head())


Перші кілька рядків даних:
    Posted On  BHK   Rent  Size            Floor    Area Type  \
0  2022-05-18    2  10000  1100  Ground out of 2   Super Area   
1  2022-05-13    2  20000   800       1 out of 3   Super Area   
2  2022-05-16    2  17000  1000       1 out of 3   Super Area   
3  2022-07-04    2  10000   800       1 out of 2   Super Area   
4  2022-05-09    2   7500   850       1 out of 2  Carpet Area   

              Area Locality     City Furnishing Status  Tenant Preferred  \
0                    Bandel  Kolkata       Unfurnished  Bachelors/Family   
1  Phool Bagan, Kankurgachi  Kolkata    Semi-Furnished  Bachelors/Family   
2   Salt Lake City Sector 2  Kolkata    Semi-Furnished  Bachelors/Family   
3               Dumdum Park  Kolkata       Unfurnished  Bachelors/Family   
4             South Dum Dum  Kolkata       Unfurnished         Bachelors   

   Bathroom Point of Contact  
0         2    Contact Owner  
1         1    Contact Owner  
2         1    Contact Owner  
3

#### Перевірка на наявність пропущених значень

Далі перевіримо, чи є в наборі даних пропущені значення. Це важливо, оскільки пропущені значення можуть вплинути на точність моделі прогнозування.

In [171]:
# Перевірка на наявність пропущених значень
missing_values = df.isnull().sum()

# Виведення кількості пропущених значень для кожного стовпчика
print("\nКількість пропущених значень у кожному стовпчику:")
print(missing_values)


Кількість пропущених значень у кожному стовпчику:
Posted On            0
BHK                  0
Rent                 0
Size                 0
Floor                0
Area Type            0
Area Locality        0
City                 0
Furnishing Status    0
Tenant Preferred     0
Bathroom             0
Point of Contact     0
dtype: int64


Згідно з результатами, в наборі даних пропущених значеннь на виявлено.

У разі, якщо пропущені дані було знайдено, можна було б:
1. Заповнити відсутні комірки (для числових даних):
   - середніми значеннями,
   - медіаною,
   - або модою.
2. Передбачити їх на основі інших змінних.
3. Видалити рядки з пропущеними значеннями. 

#### Описова статистика та аналіз розподілів

Переглянемо основні статистичні показники для кожного стовпчика даних. Це дозволить зрозуміти діапазони значень, середні, дисперсії та стандартні відхилення. Ці дані допоможуть зрозуміти розподіли числових змінних, знайти можливі аномалії або невідповідності, а також побачити, які змінні є числовими, а які — категоріальними.

In [172]:
# Перевірка типів даних
print("\nТипи даних у наборі:")
print(df.dtypes)

# Описова статистика для числових стовпців
print("\nОписова статистика для числових стовпців:")
print(df.describe())


Типи даних у наборі:
Posted On            object
BHK                   int64
Rent                  int64
Size                  int64
Floor                object
Area Type            object
Area Locality        object
City                 object
Furnishing Status    object
Tenant Preferred     object
Bathroom              int64
Point of Contact     object
dtype: object

Описова статистика для числових стовпців:
               BHK          Rent         Size     Bathroom
count  4746.000000  4.746000e+03  4746.000000  4746.000000
mean      2.083860  3.499345e+04   967.490729     1.965866
std       0.832256  7.810641e+04   634.202328     0.884532
min       1.000000  1.200000e+03    10.000000     1.000000
25%       2.000000  1.000000e+04   550.000000     1.000000
50%       2.000000  1.600000e+04   850.000000     2.000000
75%       3.000000  3.300000e+04  1200.000000     2.000000
max       6.000000  3.500000e+06  8000.000000    10.000000


### Ковертація даних у формат datetime

Звернемо увагу на те, що колонка "Posted On" у наборі даних містить дати і має тип `object`. Дані про дату публікації можуть бути корисними для аналізу, наприклад, для виявлення сезонних трендів або визначення того, як довго оголошення про оренду перебувало на ринку. Щоб уникнути помилок під час обробки даних і підготувати їх для подальшого аналізу, необхідно конвертувати цю колонку у формат datetime. Ось деякі переваги такого перетворення:
1. **Обробка дат:** Формат datetime дозволяє виконувати різні операції з датами, такі як обчислення різниці в днях між датами, групування даних за періодами тощо. Це надає можливість більш детального аналізу, ніж просте зберігання дат як рядків.
2. **Машинне навчання:** Дати у форматі datetime можуть бути корисними під час побудови моделей, оскільки деякі алгоритми краще працюють з датами, представленими у числовій формі (наприклад, кількість днів від певної дати).
3. **Уникнення помилок:** Конвертація до числового формату допоможе уникнути помилок типу ValueError: could not convert string to float, що виникають під час спроби обробити текстові дані як числові.

In [173]:
# Перетворюємо колонку "Posted On" у формат datetime
df['Posted On'] = pd.to_datetime(df['Posted On'], format='%Y-%m-%d')

# Перевірка типів даних
print("\nТипи даних у наборі після перетворення:")
print(df.dtypes)

# Виведення перших 5 рядків (за замовчуванням) набору даних
print("Перші кілька рядків даних після перетворення:")
print(df.head())


Типи даних у наборі після перетворення:
Posted On            datetime64[ns]
BHK                           int64
Rent                          int64
Size                          int64
Floor                        object
Area Type                    object
Area Locality                object
City                         object
Furnishing Status            object
Tenant Preferred             object
Bathroom                      int64
Point of Contact             object
dtype: object
Перші кілька рядків даних після перетворення:
   Posted On  BHK   Rent  Size            Floor    Area Type  \
0 2022-05-18    2  10000  1100  Ground out of 2   Super Area   
1 2022-05-13    2  20000   800       1 out of 3   Super Area   
2 2022-05-16    2  17000  1000       1 out of 3   Super Area   
3 2022-07-04    2  10000   800       1 out of 2   Super Area   
4 2022-05-09    2   7500   850       1 out of 2  Carpet Area   

              Area Locality     City Furnishing Status  Tenant Preferred  \
0     

#### Перетворення бінарних змінних

Спробуємо знайти в наборі даних бінарні змінні та перетворити їх на числові. Це може бути корисно для подальшого аналізу та моделювання.
Для цього переглянемо унікальні значення кожного стовпчика та визначимо, які з них можна вважати бінарними.

In [174]:
df.nunique()

Posted On              81
BHK                     6
Rent                  243
Size                  615
Floor                 480
Area Type               3
Area Locality        2235
City                    6
Furnishing Status       3
Tenant Preferred        3
Bathroom                8
Point of Contact        3
dtype: int64

Як бачимо, в наборі даних немає бінарних змінних, але є ознаки, що мають невелику кількість унікальних значеннь.
Розглянемо їх окремо та вирішимо, як їх краще перетворити для подальшого аналізу.

##### *Ознаки з **трьома** унікальними значеннями*

Наступні ознаки мають три унікальних значення:

- **Area Type**
- **Furnishing Status**
- **Tenant Preferred**
- **Point of Contact**

Кожна з цих ознак може мати вплив на цільову змінну "Rent", тому їх слід перетворити на числові для подальшого аналізу та моделювання.
Ці ознаки можуть бути зведені до бінарних значень або закодовані за допомогою методу "one-hot encoding". Наприклад, ознаку **Furnishing Status** можна перетворити на три бінарні змінні: `Is_Unfurnished`, `Is_Semi_Furnished`, `Is_Furnished`. Це дозволить ефективніше працювати з цими змінними в моделях машинного навчання, зокрема, у регресійних моделях, де потрібно обробляти категоріальні змінні як числові.

**One-Hot Encoding**: Це метод перетворення категоріальних змінних у числові, при якому кожне унікальне значення категоріальної змінної перетворюється в окрему бінарну змінну. Цей підхід дозволяє уникнути порядкової інтерпретації категорій, що може бути корисним у моделях машинного навчання.

Щоб застосувати метод "One-Hot Encoding" до, ви скористаємось функцією `pd.get_dummies()` із бібліотеки `pandas`. Цей метод дозволяє перетворити категоріальні змінні на бінарні, де кожна категорія перетворюється в окрему колонку.

Зауважимо, що опція `drop_first=True` вказує на те, щоб уникнути пастки фіктивних змінних ("dummy variable trap"), тобто автоматично видалити одну з нових бінарних колонок для кожної категоріальної змінної. Це знижує [**мультиколінеарність**](https://uk.wikipedia.org/wiki/Мультиколінеарність) у регресійних моделях.

In [175]:
# Вибираємо категоріальні колонки, до яких хочемо застосувати One-Hot Encoding
categorical_columns = ['Area Type', 'Furnishing Status', 'Tenant Preferred', 'Point of Contact']

# Застосовуємо One-Hot Encoding до обраних колонок
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

##### *Перетворення ознаки **"City"***

Ознака "City" має 6 унікальних значень. Це категоріальна змінна, яка без сумніву впливає на цільову змінну "Rent". Щоб врахувати цю ознаку у моделі, її також слід перетворити на числові значення. Ми отримаємо 6 нових бінарних змінних, кожна з яких — це назва міста. Це дозволить моделі розрізняти між різними містами та їх впливом на цільову змінну.

In [176]:
# Спершу переглянемо унікальні значення у колонці 'City'
print(df['City'].unique())

# Застосування One-Hot Encoding до колонки 'City'
df = pd.get_dummies(df, columns=['City'], drop_first=True)

['Kolkata' 'Mumbai' 'Bangalore' 'Delhi' 'Chennai' 'Hyderabad']


##### *Перетворення ознаки **"Floor"***

Як ми бачили раніше, ознака "Floor" містить 480 унікальних значень. Аналіз цих значень показує, що вони містять різноманітні формати запису, включаючи номери поверхів, спеціальні позначення ("Ground", "Upper Basement", "Lower Basement"), та відношення (наприклад, "5 out of 12" означає, що приміщення розташоване на 5-му поверсі в будівлі з 12 поверхами). 

Для подальшої обробки цієї колонки можна застосувати такі підходи:

1. **Створення двох окремих колонок**

Розділимо колонку на дві частини:

- **"Floor_Number"**: Номер поверху, на якому знаходиться приміщення.
- **"Total_Floors"**: Загальна кількість поверхів у будівлі.

Для цього напишемо код, який виділяє числові значення з рядка і перетворює їх у відповідні колонки. Якщо поверх позначений як "Ground", "Upper Basement", або "Lower Basement", можна закодувати ці значення як -1, 0 та -2 відповідно для "Floor_Number".

2. **Перетворення категоріальних значень в числові**

Такі значення, як "Ground", "Upper Basement", "Lower Basement", можна перетворити на числові:

- "Ground" = 0
- "Upper Basement" = -1
- "Lower Basement" = -2

Це дасть змогу використати ці значення в моделях, що вимагають числових даних.

3. **Заповнення пропущених даних**

Наявність порожніх значень у DataFrame може призвести до проблем у подальшому аналізі та моделюванні. Заповнення пропущених даних — важлива частина підготовки даних, і те, як ми це зробимо, може значно вплинути на результати моделювання.

Часто для заповнення пропущених значень використовують такі методи, як середнє значення, медіана, мода, або значення, що зустрічається найчастіше. Однак у нашому випадку, для даних про поверхи, такий підхід може бути неоптимальним і нелогічним. 

Натомість використаємо наступний підхід:
- Якщо не вдалось знайти значення для Floor_Number, можна задати значення 0, що буде означати, що поверх невідомий або що це приміщення на землі, а не у багатоповерхівці.
- У випадку, якщо немає інформації про кількість поверхів, можна прирівняти значення Total_Floors до значення Floor_Number. Це буде означати, що відсутність інформації прирівнюється до того, що цей поверх є єдиним у будинку.

Код для описаних та додаткових операцій для підготовки даних ознаки "Floor" наведено нижче.
Після виконання цих операцій ми отримаємо дві нові колонки: "Floor_Number" та "Total_Floors", які можна використовувати для подальшого аналізу та моделювання замість оригінальної колонки "Floor", яку можна видалити без втрати інформації.
Для перевірки результатів виведемо випадкові 25 рядків нового DataFrame.

In [177]:
# Функція для розділення значень на номер поверху та загальну кількість поверхів
def split_floor(floor_value):
    if "Ground" in floor_value:
        if "out of" in floor_value:
            return 0, int(floor_value.split(" out of ")[-1])
        else:
            return 0, 1  # Якщо немає інформації про кількість поверхів, вважаємо, що це єдиний поверх
    elif "Upper Basement" in floor_value:
        return -1, int(floor_value.split(" out of ")[-1])
    elif "Lower Basement" in floor_value:
        return -2, int(floor_value.split(" out of ")[-1])
    elif "out of" in floor_value:
        return int(floor_value.split(" out of ")[0]), int(floor_value.split(" out of ")[-1])
    else:
        try:
            floor_num = int(floor_value)
            return floor_num, floor_num  # Якщо немає інформації про кількість поверхів, прирівнюємо до номера поверху
        except ValueError:
            return 0, 0  # Якщо значення взагалі не вдалося знайти, задаємо 0 як індикатор невідомих даних

# Застосування функції до колонки "Floor"
df['Floor_Number'], df['Total_Floors'] = zip(*df['Floor'].map(split_floor))

# Виведення випадкових 25 рядків для перевірки
print(df[['Floor', 'Floor_Number', 'Total_Floors']].sample(25))

# Видалення колонки "Floor"
df = df.drop(columns=['Floor'])

                Floor  Floor_Number  Total_Floors
2926       1 out of 4             1             4
2898       2 out of 4             2             4
357        1 out of 2             1             2
348   Ground out of 1             0             1
3054       1 out of 4             1             4
2280       1 out of 3             1             3
1311       2 out of 4             2             4
3039       3 out of 4             3             4
4637       1 out of 2             1             2
684        7 out of 8             7             8
2278       2 out of 4             2             4
597       3 out of 30             3            30
929        5 out of 5             5             5
1500       1 out of 4             1             4
1474       1 out of 2             1             2
4611       2 out of 3             2             3
825      14 out of 20            14            20
2390  Ground out of 4             0             4
751       9 out of 14             9            14


##### *Перетворення ознаки **"Area Locality"***

Фраза, яку часто приписують британському магнату нерухомості Гарольда Семюеля, стверджує:

***"У нерухомості є три найважливіші речі: розташування, розташування і розташування."***


In [178]:
# Перетворюємо всі колонки, які містять значення True/False або результат get_dummies, на 0 і 1
# Ми перевіряємо тип даних і застосовуємо astype(int) тільки до відповідних колонок
for column in df.columns:
    if df[column].dtype == 'bool':
        df[column] = df[column].astype(int)

# Перевіряємо перші кілька рядків перетвореного DataFrame
print(df.head())

   Posted On  BHK   Rent  Size             Area Locality  Bathroom  \
0 2022-05-18    2  10000  1100                    Bandel         2   
1 2022-05-13    2  20000   800  Phool Bagan, Kankurgachi         1   
2 2022-05-16    2  17000  1000   Salt Lake City Sector 2         1   
3 2022-07-04    2  10000   800               Dumdum Park         1   
4 2022-05-09    2   7500   850             South Dum Dum         1   

   Area Type_Carpet Area  Area Type_Super Area  \
0                      0                     1   
1                      0                     1   
2                      0                     1   
3                      0                     1   
4                      1                     0   

   Furnishing Status_Semi-Furnished  Furnishing Status_Unfurnished  ...  \
0                                 0                              1  ...   
1                                 1                              0  ...   
2                                 1                  

#### Перевірка кореляцій між змінними

Перевіримо, які змінні мають сильні кореляції з метою прогнозування (змінна, що відповідає за ціну оренди нерухомості).
Кореляційний аналіз допомагає ідентифікувати ознаки, які можуть мати найбільший вплив на цільову змінну. Це корисно для майбутнього вибору змінних для моделі.

In [180]:
# Перевірка типів даних
print("\nТипи даних у наборі:")
print(df.dtypes)


Типи даних у наборі:
Posted On                            datetime64[ns]
BHK                                           int64
Rent                                          int64
Size                                          int64
Area Locality                                object
Bathroom                                      int64
Area Type_Carpet Area                         int64
Area Type_Super Area                          int64
Furnishing Status_Semi-Furnished              int64
Furnishing Status_Unfurnished                 int64
Tenant Preferred_Bachelors/Family             int64
Tenant Preferred_Family                       int64
Point of Contact_Contact Builder              int64
Point of Contact_Contact Owner                int64
City_Chennai                                  int64
City_Delhi                                    int64
City_Hyderabad                                int64
City_Kolkata                                  int64
City_Mumbai                               